In [124]:
from datetime import datetime, timedelta
from pyspark import SparkContext, SQLContext
from pyspark.sql.functions import hour, mean
import pyspark
import random
sc = SparkContext.getOrCreate()

In [132]:
spark = pyspark.sql.SparkSession.builder.appName("auctions").getOrCreate()
df_auctions = spark.read.csv('concatenate_installs_auctions.csv', header=True)
rdd_auctions = df_auctions.rdd

rdd_auctions.take(5)

[Row(_c0='0', date='2019-04-24 06:23:29.495', device_id='4716708407362582887', is_install='True'),
 Row(_c0='1', date='2019-04-24 02:06:01.032', device_id='7143568733100935872', is_install='True'),
 Row(_c0='2', date='2019-04-20 10:15:36.274', device_id='5230323462636548010', is_install='True'),
 Row(_c0='3', date='2019-04-20 21:56:47.151', device_id='5097163995161606833', is_install='True'),
 Row(_c0='4', date='2019-04-20 22:40:41.239', device_id='6328027616411983332', is_install='True')]

In [133]:
rdd_auctions.take(10)

[Row(_c0='0', date='2019-04-24 06:23:29.495', device_id='4716708407362582887', is_install='True'),
 Row(_c0='1', date='2019-04-24 02:06:01.032', device_id='7143568733100935872', is_install='True'),
 Row(_c0='2', date='2019-04-20 10:15:36.274', device_id='5230323462636548010', is_install='True'),
 Row(_c0='3', date='2019-04-20 21:56:47.151', device_id='5097163995161606833', is_install='True'),
 Row(_c0='4', date='2019-04-20 22:40:41.239', device_id='6328027616411983332', is_install='True'),
 Row(_c0='5', date='2019-04-20 18:19:27.485', device_id='7522785771858684314', is_install='True'),
 Row(_c0='6', date='2019-04-20 03:40:21.239', device_id='7882044913917355073', is_install='True'),
 Row(_c0='7', date='2019-04-23 20:55:27.691', device_id='404154463572960372', is_install='True'),
 Row(_c0='8', date='2019-04-23 18:14:14.756', device_id='1448747284042458525', is_install='True'),
 Row(_c0='9', date='2019-04-23 06:32:40.906', device_id='1465267208368809184', is_install='True')]

In [134]:
def str_to_boolean(string):
    return False if string == "False" else True

In [135]:
rdd_auctions= rdd_auctions.map(lambda x: (x[2], (datetime.strptime(x[1], "%Y-%m-%d %H:%M:%S.%f"), str_to_boolean(x[3]))))

In [136]:
rdd_auctions.take(1)

[('4716708407362582887',
  (datetime.datetime(2019, 4, 24, 6, 23, 29, 495000), True))]

In [137]:
#ME QUEDO solo con los 3 primeros dias para uqe la distancia maxima sea de 3 dias
#arranca el 2019-04-18 00:00:00 asi que el date limite es 2019-04-20 23:59:59
limit_date_train = datetime(2019, 4, 20, 23, 59, 59, 999)
#uso los siguientes 3 dias para testear
limit_date_test_begin = datetime(2019, 4, 21, 0, 0, 0, 0)
limit_date_test_end = datetime(2019, 4, 23, 23, 59, 59, 999)

In [138]:
#filtro por los que son los primeros 3 dias para el train
rdd_auctions_train = rdd_auctions.filter(lambda x: x[1][0] < limit_date_train)
rdd_auctions_test = rdd_auctions.filter(lambda x: x[1][0] > limit_date_test_begin and x[1][0] < limit_date_test_end)

In [139]:
def is_none(value):
    return True if value is not None else False

In [20]:
def didConvert():
    return bool(random.getrandbits(1))

In [34]:
rdd_auctions_train.take(5)

[('5230323462636548010',
  (datetime.datetime(2019, 4, 20, 10, 15, 36, 274000), 'True')),
 ('5097163995161606833',
  (datetime.datetime(2019, 4, 20, 21, 56, 47, 151000), 'True')),
 ('6328027616411983332',
  (datetime.datetime(2019, 4, 20, 22, 40, 41, 239000), 'True')),
 ('7522785771858684314',
  (datetime.datetime(2019, 4, 20, 18, 19, 27, 485000), 'True')),
 ('7882044913917355073',
  (datetime.datetime(2019, 4, 20, 3, 40, 21, 239000), 'True'))]

In [140]:
rdd_train = rdd_auctions_train.groupByKey().mapValues(list).mapValues(sorted)
rdd_test = rdd_auctions_test.groupByKey().mapValues(list).mapValues(sorted)

In [106]:
rdd_train.take(20)

[('8323952185725734415',
  [(datetime.datetime(2019, 4, 19, 21, 29, 43, 34000), 'True'),
   (datetime.datetime(2019, 4, 20, 5, 35, 45, 651502), 'False'),
   (datetime.datetime(2019, 4, 20, 5, 40, 46, 744391), 'False'),
   (datetime.datetime(2019, 4, 20, 5, 48, 22, 167361), 'False'),
   (datetime.datetime(2019, 4, 20, 5, 51, 43, 749231), 'False'),
   (datetime.datetime(2019, 4, 20, 5, 51, 52, 533312), 'False')]),
 ('7301194916031390155',
  [(datetime.datetime(2019, 4, 18, 19, 31, 14, 235010), 'False'),
   (datetime.datetime(2019, 4, 18, 22, 50, 14, 743995), 'False'),
   (datetime.datetime(2019, 4, 18, 22, 50, 15, 533878), 'False'),
   (datetime.datetime(2019, 4, 19, 16, 52, 9, 152000), 'True')]),
 ('7122583905705271595',
  [(datetime.datetime(2019, 4, 19, 16, 20, 9, 790000), 'True')]),
 ('6925185306188918968',
  [(datetime.datetime(2019, 4, 18, 3, 7, 57, 97324), 'False'),
   (datetime.datetime(2019, 4, 18, 3, 8, 13, 654140), 'False'),
   (datetime.datetime(2019, 4, 18, 3, 8, 30, 429718)

In [141]:
def get_installation_timeDeltas(dateList):
    """
    esta funcion recibe una lista de fechas ordenada de las apariciones de un ID
    la idea es devolver una lista de tuplas, en la que cada tupla sea asi:
    valor 1 = fecha en que aparecio en un auction
    valor 2 = distancia temporal hasta la siguiente fecha donde convirtió (0 segundos: si convirtió en esa fecha,
                                                                            3 días (en segundos): si no convirtió 
                                                                            en una fecha posterior)
    Esta tupla solo se inserta en la lista si en la fecha donde se esta parado no hubo una instalación
    """
    distancias = []
    longitud_actual = len(dateList)
    """
    En lastConversionIndexFound guardo el indice de la última conversión encontrada, 
    no quiero iterar n veces si aplica para otras fechas
    """
    lastConversionIndexFound = None
    for x in range(longitud_actual):
        if(dateList[x][1] == True):
            continue
        if(lastConversionIndexFound is not None and dateList[x][1] == False and x <= lastConversionIndexFound):
            distancias.append([dateList[x][0],
                                    (dateList[lastConversionIndexFound][0] - dateList[x][0]).total_seconds()])
            continue
        for y in range(x, longitud_actual):
            if(y == longitud_actual - 1 and dateList[y][1] != True):
                distancias.append([dateList[x][0],
                                            timedelta(days=3).total_seconds()
                                        ])
                break
            elif(dateList[y][1] == True):
                distancias.append([dateList[x][0],
                                    (dateList[y][0] - dateList[x][0]).total_seconds()
                                    ])
                lastConversionIndexFound = y
                break
        
    return distancias

In [142]:
rdd_train = rdd_train.flatMapValues(get_installation_timeDeltas)
rdd_test = rdd_test.flatMapValues(get_installation_timeDeltas)


In [149]:
rdd_train.take(10)

[('8323952185725734415',
  datetime.datetime(2019, 4, 20, 5, 35, 45, 651502),
  259200.0),
 ('8323952185725734415',
  datetime.datetime(2019, 4, 20, 5, 40, 46, 744391),
  259200.0),
 ('8323952185725734415',
  datetime.datetime(2019, 4, 20, 5, 48, 22, 167361),
  259200.0),
 ('8323952185725734415',
  datetime.datetime(2019, 4, 20, 5, 51, 43, 749231),
  259200.0),
 ('8323952185725734415',
  datetime.datetime(2019, 4, 20, 5, 51, 52, 533312),
  259200.0),
 ('7301194916031390155',
  datetime.datetime(2019, 4, 18, 19, 31, 14, 235010),
  76854.91699),
 ('7301194916031390155',
  datetime.datetime(2019, 4, 18, 22, 50, 14, 743995),
  64914.408005),
 ('7301194916031390155',
  datetime.datetime(2019, 4, 18, 22, 50, 15, 533878),
  64913.618122),
 ('6925185306188918968',
  datetime.datetime(2019, 4, 18, 3, 7, 57, 97324),
  125512.635676),
 ('6925185306188918968',
  datetime.datetime(2019, 4, 18, 3, 8, 13, 654140),
  125496.07886)]

In [143]:
#los mapeo para pasar a csv 
rdd_train = rdd_train.map(lambda x: (x[0], x[1][0], x[1][1]))
rdd_test = rdd_test.map(lambda x: (x[0], x[1][0], x[1][1]))

In [144]:
def toCSVLine(data):
  return ','.join(str(d) for d in data)


In [145]:
#paso a csv. el repartition es para que se cree 1 solo archivo. sino me crea 1 archivo por particion
lines_train = rdd_train.map(toCSVLine)
lines_train.repartition(1).saveAsTextFile('convertion_deltas_train.csv')

In [148]:
lines_test = rdd_test.map(toCSVLine)
lines_test.repartition(1).saveAsTextFile('convertion_deltas_test.csv')

In [110]:
df_prueba = spark.read.csv('auctions.csv', header=True).limit(10000)


In [111]:
rdd_prueba = df_prueba.rdd


In [54]:
rdd_prueba.take(5)

[Row(date='2019-04-23 18:58:00.842116', device_id='2564673204772915246', ref_type_id='1', source_id='0'),
 Row(date='2019-04-23 18:58:01.530771', device_id='4441121667607578179', ref_type_id='7', source_id='0'),
 Row(date='2019-04-23 18:58:01.767562', device_id='7721769811471055264', ref_type_id='1', source_id='0'),
 Row(date='2019-04-23 18:58:02.363468', device_id='6416039086842158968', ref_type_id='1', source_id='0'),
 Row(date='2019-04-23 18:58:02.397559', device_id='1258642015983312729', ref_type_id='1', source_id='0')]

In [112]:
rdd_prueba = rdd_prueba.map(lambda x: (x[1], (datetime.strptime(x[0], "%Y-%m-%d %H:%M:%S.%f"), didConvert())))

In [113]:
rdd_prueba.take(5)

[('2564673204772915246',
  (datetime.datetime(2019, 4, 23, 18, 58, 0, 842116), False)),
 ('4441121667607578179',
  (datetime.datetime(2019, 4, 23, 18, 58, 1, 530771), False)),
 ('7721769811471055264',
  (datetime.datetime(2019, 4, 23, 18, 58, 1, 767562), False)),
 ('6416039086842158968',
  (datetime.datetime(2019, 4, 23, 18, 58, 2, 363468), True)),
 ('1258642015983312729',
  (datetime.datetime(2019, 4, 23, 18, 58, 2, 397559), True))]

In [114]:
rdd_prueba = rdd_prueba.groupByKey().mapValues(list).mapValues(sorted)

In [115]:
rdd_prueba = rdd_prueba.flatMapValues(get_installation_timeDeltas)

In [116]:
rdd_prueba.take(50)

[('2564673204772915246',
  [datetime.datetime(2019, 4, 23, 18, 48, 20, 500673), 259200.0]),
 ('2564673204772915246',
  [datetime.datetime(2019, 4, 23, 18, 58, 0, 842116), 259200.0]),
 ('4441121667607578179',
  [datetime.datetime(2019, 4, 23, 18, 58, 1, 530771), 259200.0]),
 ('4441121667607578179',
  [datetime.datetime(2019, 4, 23, 19, 8, 10, 57648), 259200.0]),
 ('4441121667607578179',
  [datetime.datetime(2019, 4, 23, 19, 18, 9, 722451), 259200.0]),
 ('7721769811471055264',
  [datetime.datetime(2019, 4, 23, 18, 58, 1, 767562), 259200.0]),
 ('7721769811471055264',
  [datetime.datetime(2019, 4, 23, 19, 50, 40, 208816), 259200.0]),
 ('6707090658317158573',
  [datetime.datetime(2019, 4, 23, 20, 58, 10, 905109), 259200.0]),
 ('8869722088125970841',
  [datetime.datetime(2019, 4, 23, 19, 7, 49, 671153), 259200.0]),
 ('7445213948764639634',
  [datetime.datetime(2019, 4, 23, 18, 58, 3, 48872), 259200.0]),
 ('2932617030932207332',
  [datetime.datetime(2019, 4, 23, 18, 58, 3, 285788), 259200.0])

In [34]:
timedelta(days=3).total_seconds()

259200.0